In [1]:
import matplotlib.pyplot as plt
import os
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
import torch
from model.res_net import Res_Net
import torch.nn as nn
import torch.optim as optim
import argparse
import numpy as np
import random

In [ ]:
dataset_train = datasets.CIFAR10(
    root='D:/User/Study/data',
    train=True,
    download=True,
    transform=ToTensor(),
)
# CIFAR10 validation dataset.
dataset_valid = datasets.CIFAR10(
    root='D:/User/Study/data',
    train=False,
    download=True,
    transform=ToTensor(),
)
data, label = dataset_train[1]
data.shape

In [ ]:
plt.style.use('ggplot')
def get_data(batch_size=64):
    # CIFAR10 training dataset.
    dataset_train = datasets.CIFAR10(
        root='D:/User/Study/data',
        train=True,
        download=True,
        transform=ToTensor(),
    )
    # CIFAR10 validation dataset.
    dataset_valid = datasets.CIFAR10(
        root='D:/User/Study/data',
        train=False,
        download=True,
        transform=ToTensor(),
    )
    # Create data loaders.
    train_loader = DataLoader(
        dataset_train, 
        batch_size=batch_size,
        shuffle=True
    )
    valid_loader = DataLoader(
        dataset_valid, 
        batch_size=batch_size,
        shuffle=False
    )
    return train_loader, valid_loader

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


In [3]:
torch.cuda.is_available()

False

In [ ]:
train_set, valid_set = get_data()

for i, data in enumerate(train_set):
    image, label = data
    image = image.to(device)
    print(image.shape, label[1])
    plt.imshow(image[1].T)
    plt.show()
    if i >= 0:
        break

In [ ]:
def study(model, trainloader, optimizer, losses, device):
    model.train()
    train_loss = 0
    train_correct = 0
    # counter = 0
    print ("@@@ Start train model @@@")
    for i, data in tqdm(enumerate(trainloader), total=len(trainloader)):
        # counter += 1
        image, label = data
        image = image.to(device)
        label =label.to(device)
        # print(image.shape)
        # print(label.shape)
        optimizer.zero_grad()
        label_pre = model(image)
        # print(label_pre.shape)
        loss = losses(label_pre, label)
        train_loss += loss.item()
        # print(train_loss)
        _, preds = torch.max(label_pre.data, 1)
        train_correct += (preds == label).sum().item()
        loss.backward()
        optimizer.step()
        # print(preds)
    epoch_loss = train_loss / len(trainloader)
    epoch_acc = 100. * (train_correct / len(trainloader.dataset))
    return epoch_loss, epoch_acc


In [ ]:
def validate(model, valloader, losses, device):
    val_loss = 0
    val_correct = 0

    for i, data in tqdm(enumerate(valloader), total=len(valloader)):

        image, label = data
        image = image.to(device)
        label =label.to(device)
        # print(image.shape)
        # print(label.shape)
        label_pre = model(image)
        # print(label_pre.shape)
        loss = losses(label_pre, label)
        val_loss += loss.item()
        # print(train_loss)
        _, preds = torch.max(label_pre.data, 1)
        val_correct += (preds == label).sum().item()
        # print(preds)
    epoch_loss = val_loss / len(valloader)
    epoch_acc = 100. * (val_correct / len(valloader.dataset))
    return epoch_loss, epoch_acc

In [ ]:
batch_size = 256
train_set, valid_set = get_data(batch_size=batch_size)
learning_rate = 0.01
model = Res_Net(img_channels=3, num_layers=18, num_classes=10)
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(
    p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.")

optimizer = optim.SGD(model.parameters(), lr=learning_rate)
# Loss function.
losses = nn.CrossEntropyLoss()

In [ ]:
for i in range(3):
    train_acc, train_loss = study(model, train_set, optimizer, losses, device)
    val_acc, val_loss = validate(model, valid_set, losses, device)
    print(train_acc, train_loss)
    print(val_acc, val_loss)



In [26]:
import torchvision
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
    pretrained=False
)
input_names = ["Iris"]
output_names = ["Iris Species Prediction"]

torch.onnx.export(model, X, "model.onnx", input_names=input_names, output_names=output_names)

d:\Program\anaconda\envs\caption_image\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
d:\Program\anaconda\envs\caption_image\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu